Import Carbonara's Data Tools by running the below cell (_ctrl+enter_)

In [23]:
import CarbonaraDataTools as CDT
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.io as pio
pio.renderers.default='notebook'
from glob import glob

1. Enter the path to your protein (it should be inside newFitData)

In [103]:
# <-- Run this cell to enter a name for your prediction
molPath_text = widgets.Text(
    value='newFitData/CalmodulinMix/',
    placeholder='Enter the path to your protein:',
    description='Enter the path to your protein:',
    disabled=False,
    layout={'width': '550px','height':'50px'},   
)

molPath_text.style = {'description_width': '200px','description_height': '25px'}

def molPath_handler(change):
    global molPath
    molPath = change.new
    with output:
        clear_output(wait=True)

molPath = 'newFitData/CalmodulinMix/'
# Output widget
output = widgets.Output()
molPath_text.observe(molPath_handler,names='value')
display(molPath_text)

Text(value='newFitData/CalmodulinMix/', description='Enter the path to your protein:', layout=Layout(height='5…

2. Enter the name you gave to the specific run you'd like to analyse

In [104]:
# <-- Run this cell to enter a name for this run
runName_text = widgets.Text(
    value='fitdata',
    placeholder='Enter the name of the run you want to analyse:',
    description='Enter the name of the run you want to analyse:',
    disabled=False,
    layout={'width': '450px','height':'50px'},   
)

runName_text.style = {'description_width': '300px','description_height': '25px'}

def runName_handler(change):
    global run_name
    run_name = change.new
    with output:
        clear_output(wait=True)

run_name = 'fitdata'
# Output widget
output = widgets.Output()
runName_text.observe(runName_handler,names='value')
display(runName_text)

Text(value='fitdata', description='Enter the name of the run you want to analyse:', layout=Layout(height='50px…

3. Check the progress of your run by selecting a log file from the dropdown

In [105]:
# <-- Run this cell to view the dropdown box
logs = CDT.getLogs(molPath,run_name)


logFile_Dropdown = widgets.Dropdown(
    options = logs,
    value = logs[0],
    description = 'Select a logFile from the dropdown box ',
    disabled=False,
    layout={'width': '550px','height':'25px'}
)

# Function to display tail of DataFrame
def display_tail(log_path):
    df = CDT.LogFile2df(log_path)
    display(df.tail())

def dropdown_handler(change):
    global log_path
    log_path = change.new
    with output:
        clear_output(wait=True)  # Clear previous output
        display_tail(log_path)
        CDT.viewBestSAXSFit(molPath,log_path).show()

# Apply CSS style to the dropdown box to make it more visible
logFile_Dropdown.style = {'description_width': '250px','description_height': '25px'}

# Output widget
output = widgets.Output()

# Display initial output
log_path = logs[0]
with output:
    display_tail(log_path)
    CDT.viewBestSAXSFit(molPath,log_path).show()

logFile_Dropdown.observe(dropdown_handler,names='value')
display(logFile_Dropdown)
display(output)

Dropdown(description='Select a logFile from the dropdown box ', layout=Layout(height='25px', width='550px'), o…

Output()

In [106]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go#
import numpy as np

def load_any_coords(mol_fl_path):
    flat_coords = np.genfromtxt(mol_fl_path)
    flat_coords = flat_coords[~np.isnan(flat_coords).any(axis=1)]
    breaking_indices = CDT.missing_ca_check(flat_coords)
    if len(breaking_indices) > 0:
        chains = [i+1 for i in range(len(breaking_indices)+1)]
        coords_chains = CDT.break_into_chains(flat_coords,[0],breaking_indices)[0]
        return coords_chains
    else:
        return [flat_coords]

def get_best_mols(log_path):
    df = CDT.LogFile2df(log_path)
    best_mol_path = df['MoleculePath'].tail(1).values[0]
    molpaths = glob('_'.join(best_mol_path.split('_')[:2])+'_*_'+'_'.join(best_mol_path.split('_')[3:]))
    return molpaths

def plot_best_mols(log_path):
    molpaths = get_best_mols(log_path)
    no_mols = len(molpaths)
    fig = make_subplots(rows=1,
                        cols=no_mols,
                        column_widths=[round(1/no_mols, 1) for i in range(no_mols)],
                        horizontal_spacing=0,
                        specs=[[{'type': 'scene'} for _ in range(no_mols)]])
    for i in range(no_mols):
        coords_tensor = load_any_coords(molpaths[i])
        for coords in coords_tensor:
            fig.add_trace(go.Scatter3d( x=coords[:,0], 
                                        y=coords[:,1], 
                                        z=coords[:,2],
                                        marker=dict(
                                            size=1,
                                        ),
                                        line=dict(
                                            width=10
                                        )
                                    ), 
                                    row=1,
                                    col=i+1)
    for i in range(no_mols):
        scene_name = f'scene{i+1}'
        fig.update_layout(**{
            scene_name: dict(
                xaxis_title='',
                yaxis_title='',
                zaxis_title='',
                aspectratio=dict(x=1, y=1, z=1),
                aspectmode='manual',
                xaxis=dict(
                    visible=False,
                    showbackground=False,
                    showticklabels=False,
                ),
                yaxis=dict(
                    visible=False,
                    showbackground=False,
                    showticklabels=False),
                zaxis=dict(
                    visible=False,
                    showbackground=False,
                    showticklabels=False)
            )
        })
    annotations = []
    for i in range(no_mols):
        annotations.append(dict(
            x=(i+0.5) / no_mols,
            y=1.05,
            xref="paper",
            yref="paper",
            text=f'Molecule {i+1}',  # Change text as needed (e.g., molecule names)
            showarrow=False,
            font=dict(size=24)
        ))

    fig.update_layout(
        showlegend=False,
        annotations=annotations
    )
    return fig

In [107]:
plot_best_mols(log_path)

5. View the changes to your structure as we included more of the SAXs data.

In [6]:
# <-- Run this cell to view the q slider

q = CDT.getqChanges(log_path)
qrange_slider = widgets.SelectionSlider(
    options=q,
    value = q[0],
    description='Max q:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout={'width': '400px', 'height': '100px'}
    )

def slider_handler(change):
    global q_choice
    q_choice = change.new
    with output:
        clear_output(wait=False)
        saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q_choice)
        CDT.plotMolAndSAXS(molPath,saxs_fl,mol_fl).show("notebook")

display(qrange_slider)
saxs_fl, mol_fl = CDT.getSAXsandMolFile(log_path,q[0])
# Output widget
output = widgets.Output()
with output:
    CDT.plotMolAndSAXS(molPath,saxs_fl,mol_fl).show("notebook")   
qrange_slider.observe(slider_handler,names='value')
display(output)

SelectionSlider(continuous_update=False, description='Max q:', layout=Layout(height='100px', width='400px'), o…

Output()